# BAG3D BGT Intersection
This notebook computes the optimial intersection height between 3DBAG building and 3Dfied BGT buildings based on point clouds.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import trimesh
import numpy as np
import laspy

# Suppress loading warnings
import logging
logger = logging.getLogger("trimesh")
logger.setLevel(logging.ERROR)

from src.scrapers.bag_scraper import get_bag_by_tile_codes
from src.scrapers.bag3d_scraper import get_bag3d_as_json
from src.scrapers.bgt_scraper import get_bgt_by_tile_codes

from src.intersect import intersect

root = 'C:/Users/boble/Documents/AI-year2/3DBAG_BGT_Intersect'

### Load all city data-sources

In [ ]:
# Load necesarry data sources
tile_codes = ['2445_9723', '2445_9724', '2445_9725', '2446_9723', '2446_9724', '2446_9725', '2447_9724', '2447_9725', '2447_9726', '2448_9724', '2448_9725', '2448_9726', '2449_9724', '2449_9725', '2449_9726', '2450_9724', '2450_9725', '2450_9726',]

# 2.5D building model (Can take a minute)
city_model = get_bag3d_as_json(tile_codes)

# 2D city footprints
city_map = get_bgt_by_tile_codes(tile_codes, padding=5)

# 2D building outlines
city_outlines = get_bag_by_tile_codes(tile_codes, padding=5)

## Preprocess point clouds

In [ ]:
from src.utils.pointclouds import clean_pointcloud_tiles, divide_tiles_per_building

save_colors = True
voxel_size = 0.1

### Clean point clouds

In [ ]:
input_dir = 'D:/datasets/Amsterdam/datalab_env/run1'
output_dir = 'D:/datasets/Amsterdam/datalab_env/cleaned'

clean_pointcloud_tiles(input_dir, output_dir, save_colors=save_colors, voxel_size=voxel_size)

### Divide pointcloud per building polygon

In [ ]:
input_dir = 'D:/datasets/Amsterdam/datalab_env/cleaned'
output_dir = 'D:/datasets/Amsterdam/buildings'

buffer_inside = -0.5
buffer_outside = 1.0

divide_tiles_per_building(input_dir, output_dir, city_map, city_outlines, buffer_inside, buffer_outside, save_colors)

## Compute optimal intersection height
And save the results as `.obj`

### Parameter explanation
* `idx`: List of strings containing building idx (bag_ids) for the dataloader.
* `out_folder`: Output folder.
* `dataset='Amsterdam'`: Switch for different datasets.
* `stepsize=0.1`: Defines the interval to check the intersection. Defaults to 0.1 meters.
* `N=10000`: Number of evaluation samples. Higher is more accurate but slower.
* `improvement_threshold=0.0`: Only use the new building model when the improvement is higher then this threshold.
* `bottom_buffer=2.5`: Only allow intersections above this buffer with respect to the 3dbag bottom.
* `top_buffer=2.5`: Only allow intersections bellow this buffer with respect to the 3dbag top.
* `smooth=True`: Add smoothing to reduce the impact of outliers.

In [ ]:
# List of strings containing building idx (bag ids) for the dataloader
idx = [file[:-4] for file in os.listdir('D:/datasets/Amsterdam/buildings/')] # Complete folder
# idx = ['0363100012165490'] # One building
# idx = ['0363100012181176'] # One building

results_summary = intersect(
    idx=idx,
    out_folder=os.path.join(root, 'data', 'ply'),
    dataset_root='D:/datasets/Amsterdam/buildings',
    stepsize=0.1,
    N=10000,
    improvement_threshold=0.01,
    bottom_buffer=3,
    top_buffer=6,
    smooth=True,
    city_model=city_model,
    city_map=city_map,
    city_outline=city_outlines
    )

results_summary

## Convert `.obj` to `.cityjson`

In [ ]:
from pathlib import Path

from cjio import cityjson
from cjio.models import CityObject

from src.utils.cityjson import trimesh_to_geometry

package_dir = Path(__name__).resolve().parent.parent.parent
schema_dir = package_dir / 'cjio' / 'schemas'/ '1.1.0'
data_dir = package_dir / 'tests' / 'data'

In [ ]:
cm = cityjson.CityJSON()

In [ ]:
for file in os.listdir(os.path.join(root, 'data', 'ply')):
    if file[-4:] == '.ply':
        print(f'Processing: {file}')
        
        # Initiate object
        co = CityObject(
            id=file[:-4],
            type='Building'
        )

        # Load trimesh
        mesh = trimesh.load_mesh(os.path.join(root, 'data', 'ply', file))

        # Initiate geometry
        geom = trimesh_to_geometry(mesh)

        # Add geometry to City object
        co.geometry.append(geom)
        co.type = 'Building'
        cm.cityobjects[co.id] = co
        cm.add_to_j()

cm.update_bbox()

In [ ]:
cm.is_transformed = False
cityjson.save(cm, 'D:/test_create.json')